In [1]:
# import
import os
import glob
import psycopg2
import datetime
import openpyxl
import numpy as np
import pandas as pd

# set directory
dir = 'C:\\oasis\\git\\data\\microdata\\'

# set table name
table = 'm1.aggregate_bld'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)
    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

In [7]:
# 전체 파일 불러오기
for file in glob.glob(dir + "*집합.xlsx"): # 엑셀 하나씩 불러오기
    
    # 해당 파일의 전체 시트 가져오기
    wb = openpyxl.load_workbook(file)
    sheet_list = wb.sheetnames
    
    for sheet, sheet_len in zip(sheet_list, range(len(sheet_list))): # 시트 하나씩 불러오기
    
        df = pd.read_excel(file, sheet_name=sheet)

        # 년도마다 ID 컬럼 유무가 달라서 사용할 컬럼 선택함
        df = df[['광역상권', '상권명', '소재지', '대지면적', '건축면적', '연면적', '건폐율', '용적률', '건물구조',
           '주용도', '지상층수', '지하층수', '사용승인_준공_일자', '승강기수', '주차장면적', '용도지역', '지하철역출구',
           '버스정류장', '주요교차로', '전환률', '전통시장', '기준층', '조사층', '임차인사용현황구분', '임차인업종대분류',
           '임차인업종소분류', '전용면적(호)', '공용면적(호)', '임차인별면적합계', '소유자수(호)', '소유자형태',
           '제1월계약임대료_보증금(총액)', '제1월계약임대료_월세(총액)', '제1월계약임대료_m당월세임대료',
           '제1월시장임대료_보증금(총액)', '제1월시장임대료_월세(총액)', '제1월시장임대료_m당월세임대료', '제1월관리비총액',
           '제1월관리비실비총액', '제1월공실', '제2월계약임대료_보증금(총액)', '제2월계약임대료_월세(총액)',
           '제2월계약임대료_m당월세임대료', '제2월시장임대료_보증금(총액)', '제2월시장임대료_월세(총액)',
           '제2월시장임대료_m당월세임대료', '제2월관리비총액', '제2월관리비실비총액', '제2월공실',
           '제3월계약임대료_보증금(총액)', '제3월계약임대료_월세(총액)', '제3월계약임대료_m당월세임대료',
           '제3월시장임대료_보증금(총액)', '제3월시장임대료_월세(총액)', '제3월시장임대료_m당월세임대료', '제3월관리비총액',
           '제3월관리비실비총액', '제3월공실', '현계약개시일', '현계약기간', '최초계약개시일', '최종조사산정액_m당단가']]


        # 컬럼명 변경
        df = df.rename(columns={
              '광역상권' : 'commercial_wide_area'
            , '상권명' : 'cname'
            , '소재지' : 'sido'
            , '대지면적' : 'plat_area'
            , '건축면적' : 'arch_area'
            , '연면적' : 'tot_area'
            , '건폐율' : 'bc_rat'
            , '용적률' : 'vl_rat'
            , '건물구조' : 'building_structure'
            , '주용도' : 'main_purps_nm'
            , '지상층수' : 'ground_floor'
            , '지하층수' : 'under_ground_floor'
            , '사용승인_준공_일자' : 'useapr_day'
            , '승강기수' : 'elevator_gb'
            , '주차장면적' : 'parking_area'
            , '용도지역' : 'land_use'
            , '지하철역출구' : 'subway_distance'
            , '버스정류장' : 'bus_distance'
            , '주요교차로' : 'main_crossroad'
            , '전환률' : 'conversion_rate'
            , '전통시장' : 'traditional_market_gb'
            , '기준층' : 'typical_floor'
            , '조사층' : 'invest_floor'
            , '임차인사용현황구분' : 'tenant_usage_status'
            , '임차인업종대분류' : 'business_type_large'
            , '임차인업종소분류' : 'business_type_small'
            , '전용면적(호)' : 'net_leasable_area'
            , '공용면적(호)' : 'common_area'
            , '임차인별면적합계' : 'sum_area'
            , '소유자수(호)' : 'owner_ho'
            , '소유자형태' : 'owner_type'
            , '제1월계약임대료_보증금(총액)' : 'contract_deposit_1st'
            , '제1월계약임대료_월세(총액)' : 'contract_monthly_rent_sum_1st'
            , '제1월계약임대료_m당월세임대료' : 'contract_monthly_rent_1st'
            , '제1월시장임대료_보증금(총액)' : 'selling_deposit_1st'
            , '제1월시장임대료_월세(총액)' : 'selling_monthly_rent_sum_1st'
            , '제1월시장임대료_m당월세임대료' : 'selling_monthly_rent_1st'
            , '제1월관리비총액' : 'admin_cost_1st'
            , '제1월관리비실비총액' : 'admin_actual_cost_1st'
            , '제1월공실' : 'vacant_1'
            , '제2월계약임대료_보증금(총액)' : 'contract_deposit_2nd'
            , '제2월계약임대료_월세(총액)' : 'contract_monthly_rent_sum_2nd'
            , '제2월계약임대료_m당월세임대료' : 'contract_monthly_rent_2nd'
            , '제2월시장임대료_보증금(총액)' : 'selling_deposit_2nd'
            , '제2월시장임대료_월세(총액)' : 'selling_monthly_rent_sum_2nd'
            , '제2월시장임대료_m당월세임대료' : 'selling_monthly_rent_2nd'
            , '제2월관리비총액' : 'admin_cost_2nd'
            , '제2월관리비실비총액' : 'admin_actual_cost_2nd'
            , '제2월공실' : 'vacant_2'
            , '제3월계약임대료_보증금(총액)' : 'contract_deposit_3rd'
            , '제3월계약임대료_월세(총액)' : 'contract_monthly_rent_sum_3rd'
            , '제3월계약임대료_m당월세임대료' : 'contract_monthly_rent_3rd'
            , '제3월시장임대료_보증금(총액)' : 'selling_deposit_3rd'
            , '제3월시장임대료_월세(총액)' : 'selling_monthly_rent_sum_3rd'
            , '제3월시장임대료_m당월세임대료' : 'selling_monthly_rent_3rd'
            , '제3월관리비총액' : 'admin_cost_3rd'
            , '제3월관리비실비총액' : 'admin_actual_cost_3rd'
            , '제3월공실' : 'vacant_3'
            , '현계약개시일' : 'current_contract_date'
            , '현계약기간' : 'current_contract_term'
            , '최초계약개시일' : 'first_current_contract_date'
            , '최종조사산정액_m당단가' : 'invest_calculation_amt'
        })

        # base_year, base_quarter 추가
        df['base_quarter'] = sheet[0]
        df['base_year'] = file.split('\\')[-1][:4]

        # 특수문자, nan, type 처리
        df = df.replace({np.nan: None})
        df = df.replace("%","", regex=True)
        df['invest_calculation_amt'] = df['invest_calculation_amt'].astype('int64')
        df['selling_monthly_rent_1st'] = df['selling_monthly_rent_1st'].astype('int64')
        df['selling_monthly_rent_2nd'] = df['selling_monthly_rent_2nd'].astype('int64')
        df['selling_monthly_rent_3rd'] = df['selling_monthly_rent_3rd'].astype('int64')

        # values 설정
        val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

        # insert
        for j in range(0, len(df), 10000):
            # DB Connect
            conn = psycopg2.connect(
                host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
                port=5439,
                dbname='dev',
                user='awsuser',
                password='cremaoAdmin1234qwer!!'
            )
            # 10000개 단위로 나누기
            tmp = df[j:j + 10000]

            # insert
            execute_mogrify(conn, tmp, table, val)
            print(datetime.datetime.now(), ' : ', j)
            tmp = pd.DataFrame()

        # row count
        print(f'{len(df)} Done : {file}')

2022-12-21 10:38:20.530615  :  0
2022-12-21 10:38:43.258049  :  10000
2022-12-21 10:38:45.390699  :  20000
20474 Done : C:\oasis\git\data\microdata\2019_집합.xlsx
2022-12-21 10:39:04.604264  :  0
2022-12-21 10:39:13.663930  :  10000
2022-12-21 10:39:15.583007  :  20000
20474 Done : C:\oasis\git\data\microdata\2019_집합.xlsx
2022-12-21 10:39:35.938327  :  0
2022-12-21 10:39:45.027658  :  10000
2022-12-21 10:39:47.077908  :  20000
20474 Done : C:\oasis\git\data\microdata\2019_집합.xlsx
2022-12-21 10:40:08.231807  :  0
2022-12-21 10:40:17.750553  :  10000
2022-12-21 10:40:20.150560  :  20000
20474 Done : C:\oasis\git\data\microdata\2019_집합.xlsx
2022-12-21 10:41:22.574618  :  0
2022-12-21 10:41:32.840993  :  10000
2022-12-21 10:41:35.094198  :  20000
20474 Done : C:\oasis\git\data\microdata\2020_집합.xlsx
2022-12-21 10:41:57.711978  :  0
2022-12-21 10:42:06.795867  :  10000
2022-12-21 10:42:08.775847  :  20000
20474 Done : C:\oasis\git\data\microdata\2020_집합.xlsx
2022-12-21 10:42:28.444520  :  0
2